# CONFLUENCE Tutorial: Continental-Scale Modeling - North America

This notebook demonstrates how to set up a continental-scale SUMMA model for North America. We'll move quickly through the workflow, focusing on the scale differences from previous tutorials.

## Key Points
- **Scale**: From country (Iceland) to continent (North America)
- **Computational considerations**: Much larger domain
- **Simplified approach**: Quick setup for demonstration

## 1. Quick Setup

In [ ]:
# Import required libraries
import sys
from pathlib import Path
import yaml
import matplotlib.pyplot as plt

# Add CONFLUENCE to path
confluence_path = Path('../').resolve()
sys.path.append(str(confluence_path))

# Import CONFLUENCE
from CONFLUENCE import CONFLUENCE

plt.style.use('default')
%matplotlib inline

## 2. Configure for North America

In [ ]:
# Load and modify the North America configuration
na_config_path = confluence_path / 'config_North_America.yaml'

with open(na_config_path, 'r') as f:
    config = yaml.safe_load(f)

# Quick look at the continental configuration
print("=== North America Configuration ===")
print(f"Domain: {config['DOMAIN_NAME']}")
print(f"Bounding Box: {config['BOUNDING_BOX_COORDS']}")
print(f"Stream Threshold: {config['STREAM_THRESHOLD']} (larger for continental scale)")
print(f"Min GRU Size: {config['MIN_GRU_SIZE']} km²")
print(f"MPI Processes: {config['MPI_PROCESSES']} (high for parallel processing)")

# Parse the continental extent
bbox = config['BOUNDING_BOX_COORDS'].split('/')
lat_max, lon_min, lat_min, lon_max = map(float, bbox)
print(f"\nGeographic extent:")
print(f"  North: {lat_max}°N (Arctic)")
print(f"  South: {lat_min}°N (Central America)")
print(f"  West: {lon_min}°W (Pacific)")
print(f"  East: {lon_max}°W (Atlantic)")

## 3. Visualize Continental Scale

In [ ]:
# Simple visualization comparing scales
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(18, 6))

# Bow at Banff (Tutorial 1)
ax1.add_patch(plt.Rectangle((0.1, 0.1), 0.8, 0.8, fill=True, 
                           color='lightblue', alpha=0.7))
ax1.text(0.5, 0.5, 'Bow at Banff\n~2,200 km²', ha='center', va='center', 
         fontsize=14, fontweight='bold')
ax1.set_title('Watershed Scale', fontsize=16)
ax1.axis('off')

# Iceland (Tutorial 4)
ax2.add_patch(plt.Rectangle((0.1, 0.1), 0.8, 0.8, fill=True, 
                           color='lightgreen', alpha=0.7))
ax2.text(0.5, 0.5, 'Iceland\n~103,000 km²', ha='center', va='center', 
         fontsize=14, fontweight='bold')
ax2.set_title('Country Scale', fontsize=16)
ax2.axis('off')

# North America (This tutorial)
ax3.add_patch(plt.Rectangle((0.1, 0.1), 0.8, 0.8, fill=True, 
                           color='lightcoral', alpha=0.7))
ax3.text(0.5, 0.5, 'North America\n~24,700,000 km²', ha='center', va='center', 
         fontsize=14, fontweight='bold')
ax3.set_title('Continental Scale', fontsize=16)
ax3.axis('off')

plt.suptitle('Scale Comparison', fontsize=20, fontweight='bold')
plt.tight_layout()
plt.show()

# Show relative sizes
print("Scale factors:")
print(f"Iceland is ~47x larger than Bow at Banff")
print(f"North America is ~240x larger than Iceland")
print(f"North America is ~11,200x larger than Bow at Banff")

## 4. Initialize CONFLUENCE for Continental Domain

In [ ]:
# Initialize CONFLUENCE
confluence = CONFLUENCE(na_config_path)

print(f"Project: {confluence.config['DOMAIN_NAME']}")
print(f"Directory: {confluence.project_dir}")

## 5. Quick Workflow Execution

Let's run through the key steps quickly, noting the continental-scale considerations.

In [ ]:
# Step 1: Project setup
print("Setting up continental project...")
confluence.setup_project()
print("✓ Done\n")

# Step 2: Skip pour point (continental domain)
print("Pour point: Not needed for continental domain\n")

# Step 3: Acquire continental attributes
print("Acquiring continental-scale attributes...")
print("Note: This downloads LARGE datasets (DEM, soil, land cover)")
print("Expected data size: Several GB")
confluence.acquire_attributes()
print("✓ Done\n")

# Step 4: Define continental domain
print("Delineating continental domain...")
print(f"Stream threshold: {confluence.config['STREAM_THRESHOLD']} (high for continent)")
print("This creates thousands of watersheds...")
confluence.define_domain()
print("✓ Done\n")

# Step 5: Apply discretization
print("Creating continental HRUs...")
confluence.discretize_domain()
print("✓ Done\n")

# Quick summary of scale
print("\n=== Continental Scale Summary ===")
print(f"Domain: {confluence.config['DOMAIN_NAME']}")
print(f"Extent: {abs(lat_max - lat_min):.1f}° latitude × {abs(lon_max - lon_min):.1f}° longitude")
print(f"MPI Processes: {confluence.config['MPI_PROCESSES']}")
print("Expected: Thousands of GRUs, tens of thousands of HRUs")

## 6. Complete SUMMA Setup

In [ ]:
# Continue with forcing data and model setup
print("Completing continental SUMMA setup...\n")

# Acquire continental forcing data
print("Step 6: Acquiring continental forcing data...")
print(f"Dataset: {confluence.config['FORCING_DATASET']}")
print(f"Period: {confluence.config['FORCING_START_YEAR']}-{confluence.config['FORCING_END_YEAR']}")
print("Note: Continental forcing data can be 10s of GB")
confluence.acquire_forcings()
print("✓ Done\n")

# Model preprocessing
print("Step 7: Model-agnostic preprocessing...")
confluence.model_agnostic_pre_processing()
print("✓ Done\n")

print("Step 8: SUMMA-specific preprocessing...")
confluence.model_specific_pre_processing()
print("✓ Done\n")

print("=== Continental SUMMA Model Ready ===")
print(f"Model: {confluence.config['HYDROLOGICAL_MODEL']}")
print(f"Routing: {confluence.config['ROUTING_MODEL']}")
print("\nTo run the model:")
print("  confluence.run_models()")
print("\nNote: Continental simulations require significant compute resources!")

## 7. Continental-Scale Considerations

### Computational Requirements
- **Memory**: 10s of GB for data processing
- **Storage**: 100s of GB for inputs/outputs
- **CPU**: High parallelization (40+ cores)
- **Runtime**: Days to weeks for full simulations

### Key Configuration Differences
```yaml
STREAM_THRESHOLD: 7500        # Higher for continental scale
MPI_PROCESSES: 40            # More parallel processes
MIN_GRU_SIZE: 0              # Accept all watersheds
FORCING_DATASET: ERA5        # Global reanalysis data
```

### Challenges at Continental Scale
1. **Data Volume**: TB of input/output data
2. **Heterogeneity**: Diverse climates, terrains, vegetation
3. **Calibration**: How to calibrate thousands of watersheds?
4. **Validation**: Limited observations for many basins
5. **Computation**: Requires HPC resources

### Use Cases
- Climate change impact assessment
- Continental water balance
- Large-scale flood forecasting
- Water resources planning
- Earth system modeling

In [ ]:
# Final summary
print("=== Continental Model Setup Complete ===\n")
print(f"Domain: {confluence.config['DOMAIN_NAME']}")
print(f"Scale: Continental (~24.7 million km²)")
print(f"Model: {confluence.config['HYDROLOGICAL_MODEL']}")
print(f"Status: Ready for simulation")

print("\nScale progression in tutorials:")
print("  1. Watershed: Bow at Banff (~2,200 km²)")
print("  2. Country: Iceland (~103,000 km²)")
print("  3. Continent: North America (~24,700,000 km²)")

print("\n🎉 You've successfully scaled from watershed to continent!")